# 第17章 类和方法
## 17.1 面向对象特性
Python是一门面向对象的编程语言，提供了一些支持面向对象编程的语言特性：
- 程序包括类定义和方法定义
- 大部分计算都通过对象的操作来表达
- 每个对象定义对应真实世界的某些对象或概念，而方法则对应真实世界中对象之间交互的方式

方法和函数语义上是一样的，但在语法上有两个区别。
- 方法定义写在类定义中，更明确的表示类和方法的关联
- 调用方法和调用函数的语法形式不同

## 17.2 打印对象
我们将第16章的print_time函数转换成Time类的一个方法。

In [1]:
class Time():
    def print_time(time):
        print('%.2d:%.2d:%.2d'%(time.hour,time.minute,time.second))

In [2]:
start = Time()
start.hour = 9
start.minute = 45
start.second = 00

现在有两种方式可以调用print_time，第一种方法是使用**函数调用语法**（比较少见）：

In [3]:
Time.print_time(start)

09:45:00


在这里的点表示法中，Time是类的名称，而print_time是方法的名称，start作为参数传入。<br>
另一种方式是使用**方法调用语法**。

In [4]:
start.print_time()

09:45:00


在这里的点表示法中，print_time仍是方法的名称，而start是调用这个方法的对象，也称为**主体（subject）**<br>
在方法中主体会被赋值给第一个形参，在本例中start被赋值给time。<br>

通常，方法的第一个形参应为self，所以print_time这个方法通常应该写成这样的形式：

In [5]:
class Time:
    def print_time(self):
        print('%.2d:%.2d:%.2d'%(self.hour,self.minute,self.second))
        
start.print_time()

09:45:00


## 17.5 `__init__`方法

In [6]:
class Time():
    # 新增__init__()
    def __init__(self,hour=0,minute=0,second=0):
        self.hour=hour
        self.minute=minute
        self.second=second
    
    def print_time(self):
        print('%.2d:%.2d:%.2d'%(self.hour,self.minute,self.second))

形参是可选的，所以当你不使用任何实参调用Time时，会得到默认值：

In [7]:
time = Time()
time.print_time()

00:00:00


## 17.6 `__str__`方法

`__str__`和`__init__`方法类似，它用来返回对象的字符串表达形式。

In [8]:
class Time():
    def __init__(self,hour=0,minute=0,second=0):
        self.hour=hour
        self.minute=minute
        self.second=second
    
    # 新增__str__()
    def __str__(self):
        return '%.2d:%.2d:%.2d'%(self.hour,self.minute,self.second)

In [9]:
time = Time(9,45)
print(time)

09:45:00


## 17.7 操作符重载
例如为Time类定义一个__add__方法，则可以在Time对象上使用+操作符。

In [10]:
class Time():
    def __init__(self,hour=0,minute=0,second=0):
        self.hour=hour
        self.minute=minute
        self.second=second
    
    def __str__(self):
        return '%.2d:%.2d:%.2d'%(self.hour,self.minute,self.second)
    
    def time_to_int(self):
        minutes = self.minute + self.hour * 60
        seconds = self.second + minutes * 60
        return seconds
    
    # 新增__add__()
    def __add__(self,other):
        seconds = self.time_to_int() + other.time_to_int()
        return int_to_time(seconds)

def int_to_time(seconds):
    time = Time()
    minutes , time.second = divmod(seconds,60)
    time.hour , time.minute = divmod(minutes,60)
    return time

In [11]:
start = Time(9,45)
duration = Time(1,35)
print(start + duration)

11:20:00


当你对Time对象使用+操作符时，Python会调用__add__。<br>
当你打印结果时，Python会调用__str__。<br>

修改操作符的行为以便它能够作用于用户定义类型，这个过程称为**操作符重载**。

## 17.8 基于类型的分发
在前一节中我们将两个Time对象相加，但你也可能会想要将一个Time对象加上一个整数。接下来是__add__的另一个版本，会检查other的类型，并调用add_time或者increment：

In [12]:
class Time():
    def __init__(self,hour=0,minute=0,second=0):
        self.hour=hour
        self.minute=minute
        self.second=second
    
    def __str__(self):
        return '%.2d:%.2d:%.2d'%(self.hour,self.minute,self.second)
    
    def time_to_int(self):
        minutes = self.minute + self.hour * 60
        seconds = self.second + minutes * 60
        return seconds
    
    # 修改__add__()
    def __add__(self,other):
        if isinstance(other,Time):
            return self.add_time(other)
        else:
            return self.increment(other)
        
    def add_time(self,other):
        seconds = self.time_to_int() + other.time_to_int()
        return int_to_time(seconds)
    
    def increment(self,seconds):
        seconds += self.time_to_int()
        return int_to_time(seconds)
        
def int_to_time(seconds):
    time = Time()
    minutes , time.second = divmod(seconds,60)
    time.hour , time.minute = divmod(minutes,60)
    return time

如果other是一个Time对象，`__add__`会调用`add_time`。否则认为实参是整数，并调用`increment`。这个操作称为基于**类型的分发（type-based dispatch）**，根据形参的类型，将计算分发到不同的方法上。

In [13]:
start = Time(9,45)
duration = Time(1,35)
print(start + duration)

11:20:00


In [14]:
print(start+1337)

10:07:17


这个加法的实现并不满足交换律，如果第一个操作数是整数，则会得到：

In [15]:
print(1337+start)

TypeError: unsupported operand type(s) for +: 'int' and 'Time'

这里有一个解决方案，特别方法`__radd__`，右加法（right-side add）。当Time对象出现在+号的右侧时，会调用这个方法。

In [16]:
class Time():
    def __init__(self,hour=0,minute=0,second=0):
        self.hour=hour
        self.minute=minute
        self.second=second
    
    def __str__(self):
        return '%.2d:%.2d:%.2d'%(self.hour,self.minute,self.second)
    
    def time_to_int(self):
        minutes = self.minute + self.hour * 60
        seconds = self.second + minutes * 60
        return seconds
    
    # 修改__add__()
    def __add__(self,other):
        if isinstance(other,Time):
            return self.add_time(other)
        else:
            return self.increment(other)
        
    def add_time(self,other):
        seconds = self.time_to_int() + other.time_to_int()
        return int_to_time(seconds)
    
    def increment(self,seconds):
        seconds += self.time_to_int()
        return int_to_time(seconds)
    
    def __radd__(self,other):
        return self.__add__(other)
        
def int_to_time(seconds):
    time = Time()
    minutes , time.second = divmod(seconds,60)
    time.hour , time.minute = divmod(minutes,60)
    return time

In [17]:
start = Time(9,45)
print( 1337 + start)

10:07:17


## 17.9 多态
我们编写的很多处理字符串的函数，实际上对其他序列类型也可以用。例如，在11.1节中，我们使用histogram来记录单词中每个字母出现的次数：

In [18]:
def histogram(s):
    d = dict()
    for letter in s:
        if letter not in d:
            d[letter] = 1
        else:
            d[letter] += 1
    return d

这个函数对列表、元组、字典都可以使用，只要s的元素是可散列的，即s的元素为不可变类型，如整数、浮点数和字符串。
https://www.cnblogs.com/1zhangwenjing/p/9072689.html

In [19]:
t = ['spam','egg','spam','spam','bacon','spam']
histogram(t)

{'spam': 4, 'egg': 1, 'bacon': 1}

处理多个类型的函数称为**多态（polymorphic）**，优点：可以促进代码复用<br>
例如，用来计算一个序列所有对象的和的内置函数，对所有其元素支持加法的序列都可用。<br>
由于Time对象提供了add方法，所以他们也可以使用过sum：

In [20]:
t1 = Time(7,43)
t2 = Time(7,41)
t3 = Time(7,37)
total = sum([t1,t2,t3])
print(total)

23:01:00


## 17.10 接口和实现

将接口和实现分离的设计理念，可以帮助我们提高软件的可维护性。

## 17.11 调试
如果我们并不清楚一个对象是否拥有某个属性，可以使用内置函数`hasattr()`检查一个对象是否拥有某个特定的属性。（参考15.7节）<br> 
另一种访问一个对象的属性的方法是使用内置函数`vars()`，接收一个对象，并返回一个将属性名称（字符串形式）映射到属性值的字典对象：

In [21]:
vars(t1)

{'hour': 7, 'minute': 43, 'second': 0}

为了调试，可以使用print_attributes遍历对象的属性字典，并打印出每个属性的名称和相应的值。<br>
内置函数`getattr()`接受一个对象以及一个属性名称（字符串形式）并返回属性的值。

In [22]:
def print_attributes(obj):
    for attr in vars(obj):
        print(attr,getattr(obj,attr))
        
print_attributes(t1)

hour 7
minute 43
second 0


## 17.12 术语表
- 面向对象语言（object-oriented language）
- 面向对象编程（object-oriented programming）
- 方法（method）
- 主体（subject）
- 按位实参/位置参数 （positional argument）
- 操作符重载（operator overloading）
- 基于类型的分发（type-based dispatch）
- 多态（polymorphic）
- 信息隐藏（information hiding）